In [1]:
# Importing Libraries

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset2'

In [5]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [21]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [22]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [8]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [10]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf

In [84]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [85]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [11]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [12]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [13]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [23]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

In [24]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

In [25]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32)                5376      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [26]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [27]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Epoch 1/30
460/460 [==============================] - 10s 23ms/step - loss: 1.2361 - accuracy: 0.5146 - val_loss: 0.9738 - val_accuracy: 0.6223
Epoch 2/30
460/460 [==============================] - 10s 22ms/step - loss: 0.9711 - accuracy: 0.5775 - val_loss: 0.9804 - val_accuracy: 0.5816
Epoch 3/30
460/460 [==============================] - 10s 22ms/step - loss: 0.8691 - accuracy: 0.6080 - val_loss: 0.9766 - val_accuracy: 0.5728
Epoch 4/30
460/460 [==============================] - 10s 22ms/step - loss: 0.7529 - accuracy: 0.6442 - val_loss: 0.8165 - val_accuracy: 0.6125
Epoch 5/30
460/460 [==============================] - 10s 22ms/step - loss: 0.7496 - accuracy: 0.6474 - val_loss: 0.8264 - val_accuracy: 0.6166
Epoch 6/30
460/460 [==============================] - 10s 22ms/step - loss: 0.6805 - accuracy: 0.6687 - val_loss: 0.7411 - val_accuracy: 0.6373
Epoch 7/30
460/460 [==============================] - 10s 22ms/step - loss: 0.6175 - accuracy: 0.7186 - val_loss: 0.8818 - val_accuracy:

In [28]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 510        0        22        0                   0   
SITTING                  3      394        86        1                   0   
STANDING                 0       90       442        0                   0   
WALKING                  0        0         0      477                  12   
WALKING_DOWNSTAIRS       0        0         0        1                 410   
WALKING_UPSTAIRS         0        1         1        2                   5   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                             5  
SITTING                            7  
STANDING                           0  
WALKING                            7  
WALKING_DOWNSTAIRS                 9  
WALKING_UPSTAIRS                 462  


In [29]:
score = model.evaluate(X_test, Y_test)

93/93 [==============================] - 1s 5ms/step - loss: 0.3137 - accuracy: 0.9145


In [30]:
score

[0.3137085735797882, 0.9144893288612366]

- With a simple 2 layer architecture we got 91.44% accuracy and a loss of 0.31
- We can further imporve the performace with Hyperparameter tuning